In [1]:
import requests as re
import random
import numpy as np
from math import floor

In [2]:
hero_api_key = "<HEROAPIKEY>"
class Hero:
    def __init__(self, id):
        print(f"Fetching char {id}")
        self.id = id
        self.status = 'alive'
        if self.get_hero_info(id):
            # print(self.stats)
            pass

    def get_hero_info(self, id):
        try:
            hero_info = re.get(f"https://superheroapi.com/api/{hero_api_key}/{id}").json()
            self.name = hero_info['name']
            self.base_stats = { k: int(v) for k,v in hero_info['powerstats'].items() }
            self.as_stats = { k: random.randint(0,10) for k in self.base_stats.keys() }
            self.img_url = hero_info['image']['url']
            self.alignment = hero_info['biography']['alignment']
            return True
        except ValueError as e:
            self.status = 'ERROR'
            print(f"Hero with Id {self.id} has no stats. Cannot be created")

    def _initHp(self):
        self.as_stats['hp'] = random.randint(0,10)
        self.max_hp = 100 + floor((self.new_stats['strength'] * .8 + self.new_stats['durability'] * .7 + self.new_stats['power']) / 2 * (1 + self.as_stats['hp'] / 10))
        self.hp = 100 + floor((self.new_stats['strength'] * .8 + self.new_stats['durability'] * .7 + self.new_stats['power']) / 2 * (1 + self.as_stats['hp'] / 10))
        
    def _initFB(self, teamAlignment):
        coef = 1 if teamAlignment == self.alignment else -1
        self.fb = (1 + random.randint(0,9)) ** coef
#         print(self.alignment, teamAlignment, coef, self.fb)
        self.new_stats = {k: self.fb * (2 * self.base_stats[k] + self.as_stats[k]) / 1.1 for k in self.base_stats.keys()}
        self._initHp()
        
    def attack(self, target):
        p = random.random()
        damage = 0
        if p < 1/3:
            atk_type = 'mental'
            damage = self.fb * (self.new_stats['intelligence'] * 0.7 + self.new_stats['speed'] * 0.2 + self.new_stats['combat'] * 0.1)
        elif p < 2/3:
            atk_type = 'strong'
            damage = self.fb * (self.new_stats['strength'] * 0.6 + self.new_stats['power'] * 0.2 + self.new_stats['combat'] * 0.2)
        else:
            atk_type = 'fast'
            damage = self.fb * (self.new_stats['speed'] * 0.55 + self.new_stats['durability'] * 0.25 + self.new_stats['strength'] * 0.2)
        
        target.hp = target.hp - damage
        print(f"{self.name} damages {target.name} with {atk_type} Atk for {round(damage)}. {target.name} remaining hp: {round(target.hp)}")
        if target.hp <= 0:
            print(f"{target.name} blacked out!")
            target.status = 'rip'
    
    def reset_hp(self):
        self.hp = self.max_hp
        self.status = 'alive'
        
    def __repr__(self):
        return f"{self.name} hp: {self.hp}"    

    def __str__(self):
        return f"{self.name} hp: {self.hp}"

In [3]:
class Game:
    def __init__(self):
        self.team_1 = []
        self.team_2 = []
        self.heroes = []
        self.fetch_heroes()
        self.split_teams()

    def fetch_heroes(self):
        ids = np.arange(1, 732)
        left = 10
        heroes = np.array([])
        while left > 0: 
            selectedIds = np.random.choice(ids, left, replace=False)
            heroes = np.concatenate((heroes, np.array([Hero(id) for id in selectedIds])))
            invalid = np.array([(h.id - 1, i) for i, h in enumerate(heroes) if h.status == 'ERROR'])
            left = len(invalid)
            if left != 0:
                ids = np.delete(ids, invalid[:,0])
                heroes = np.delete(heroes, invalid[:,1])
        self.heroes = heroes
        
    def split_teams(self):
        choice = np.random.choice(range(self.heroes.shape[0]), size=int(self.heroes.shape[0]/2), replace=False)    
        ind = np.zeros(self.heroes.shape[0], dtype=bool)
        ind[choice] = True
        self.team_1 = self.heroes[ind]
        self.team_2 = self.heroes[~ind]
        team_1Alignment = 'good' if np.sum([h.alignment == 'good' for h in self.team_1]) >= 3 else 'bad'
        team_2Alignment = 'good' if np.sum([h.alignment == 'good' for h in self.team_2]) >= 3 else 'bad'
        [h._initFB(team_1Alignment) for h in self.team_1]
        [h._initFB(team_2Alignment) for h in self.team_2]
            
    def simulation(self):
        nextTurn = True
        n = 0
        while nextTurn:
            n += 1
            print(f"\n\nRound {n}")
            nextTurn = self.turn()
        
    def turn(self):
        np.random.shuffle(self.team_1)
        print('Team 1 beggins their attack round')
        for h in self.team_1:
            if h.status == 'rip': continue
            target = np.random.choice(self.team_2)
            h.attack(target)
        
        if np.all([h.status == 'rip' for h in self.team_2]):
            print('\033[1m' + 'Team 1 Wins')
            send_simple_message('Congratulations!', 'won', '\U0001f451')
            return False

        print('\nTeam 2 beggins their attack round')
        for h in self.team_2:
            if h.status == 'rip': continue
            target = np.random.choice(self.team_1)
            h.attack(target)
        
        if np.all([h.status == 'rip' for h in self.team_1]):
            print('\033[1m' + 'Team 2 Wins')
            send_simple_message('Sorry!', 'lost', '\U00002620')
            return False
        return True
    
    def reset_hp(self):
        f = lambda x: x.reset_hp()
        [f(h) for h in self.heroes]

In [4]:
api_key = "<APIKEY>"
def send_simple_message(intro, result, emoji):
    return re.post(
        "https://api.mailgun.net/v3/sandboxe8205a4126134acda6b836f1262c5a27.mailgun.org/messages",
        auth=("api", api_key),
        data={"from": "Mailgun Sandbox <postmaster@sandboxe8205a4126134acda6b836f1262c5a27.mailgun.org>",
            "to": "XX YY <xx.yy@hh.zz>",
            "subject": "Battle Status",
            "text": f"{intro} XX YY, you have {result} your battle! {emoji}"})


In [5]:
send_simple_message('Sorry!', 'lost', '\U00002620')

<Response [200]>

In [6]:
game = Game()
game.split_teams()
print(*game.team_1)
print(*game.team_2)

Fetching char 464
Hero with Id 464 has no stats. Cannot be created
Fetching char 147
Fetching char 583
Fetching char 672
Fetching char 62
Fetching char 412
Fetching char 435
Fetching char 514
Fetching char 724
Fetching char 188
Fetching char 469
Cameron Hicks hp: 1272 Lizard hp: 123 Master Chief hp: 1394 Cottonmouth hp: 107 Monica Dawson hp: 758
Sentry hp: 187 Toxin hp: 1718 Bantam hp: 1288 Penguin hp: 112 X-Man hp: 1760


In [7]:
for h in game.heroes:
    print(h)
    print(h.base_stats)
    print(h.as_stats)
    print(h.new_stats)
    print(h.alignment)
    print(h.hp)
    print(h.fb, "\n")

Cameron Hicks hp: 1272
{'intelligence': 50, 'strength': 10, 'speed': 23, 'durability': 28, 'power': 51, 'combat': 70}
{'intelligence': 1, 'strength': 9, 'speed': 2, 'durability': 5, 'power': 4, 'combat': 6, 'hp': 5}
{'intelligence': 918.1818181818181, 'strength': 263.6363636363636, 'speed': 436.3636363636363, 'durability': 554.5454545454545, 'power': 963.6363636363635, 'combat': 1327.2727272727273}
good
1272
10 

Sentry hp: 187
{'intelligence': 75, 'strength': 100, 'speed': 100, 'durability': 84, 'power': 100, 'combat': 40}
{'intelligence': 4, 'strength': 4, 'speed': 3, 'durability': 0, 'power': 1, 'combat': 3, 'hp': 10}
{'intelligence': 28.0, 'strength': 37.09090909090909, 'speed': 36.90909090909091, 'durability': 30.545454545454543, 'power': 36.54545454545455, 'combat': 15.090909090909092}
neutral
187
0.2 

Toxin hp: 1718
{'intelligence': 75, 'strength': 80, 'speed': 42, 'durability': 85, 'power': 97, 'combat': 70}
{'intelligence': 0, 'strength': 7, 'speed': 8, 'durability': 0, 'powe

In [8]:
game.simulation()



Round 1
Team 1 beggins their attack round
Master Chief damages X-Man with strong Atk for 3605. X-Man remaining hp: -1845
X-Man blacked out!
Monica Dawson damages Sentry with strong Atk for 6745. Sentry remaining hp: -6558
Sentry blacked out!
Cottonmouth damages Bantam with mental Atk for 1. Bantam remaining hp: 1287
Cameron Hicks damages Penguin with strong Atk for 6164. Penguin remaining hp: -6052
Penguin blacked out!
Lizard damages Penguin with strong Atk for 2. Penguin remaining hp: -6053
Penguin blacked out!

Team 2 beggins their attack round
Toxin damages Lizard with mental Atk for 4500. Lizard remaining hp: -4377
Lizard blacked out!
Bantam damages Lizard with fast Atk for 2512. Lizard remaining hp: -6889
Lizard blacked out!


Round 2
Team 1 beggins their attack round
Master Chief damages X-Man with fast Atk for 2611. X-Man remaining hp: -4456
X-Man blacked out!
Cottonmouth damages Bantam with strong Atk for 1. Bantam remaining hp: 1286
Cameron Hicks damages X-Man with mental At

In [9]:
game.reset_hp()

In [10]:
[(h.status, h.hp, h.max_hp) for h in game.heroes]

[('alive', 1272, 1272),
 ('alive', 187, 187),
 ('alive', 1718, 1718),
 ('alive', 1288, 1288),
 ('alive', 123, 123),
 ('alive', 1394, 1394),
 ('alive', 112, 112),
 ('alive', 1760, 1760),
 ('alive', 107, 107),
 ('alive', 758, 758)]